In [19]:
library(plyr, quietly = TRUE)
library(tidyverse,  quietly = TRUE)
library(tidyjson,  quietly = TRUE)
library(readxl,  quietly = TRUE)
library(disgenet2r)
library(getPass)

In [7]:
`%ni%` <- Negate(`%in%`)

In [34]:
genesets_colnames = c("Source", "GeneSetID", "GeneSetName", "GeneList", "n_Genes")
genesets_MAT = matrix(nrow = 0, ncol = length(genesets_colnames))
colnames(genesets_MAT) = genesets_colnames
genesets_DF = as_tibble(genesets_MAT) %>%
  mutate(across(everything(), as.character))
genesets_DF$n_Genes = as.integer(genesets_DF$n_Genes)
genesets_empty_DF = genesets_DF

### NeST

In [60]:
NeST_DF = read_delim("data/Omics_data/NeST__IAS_clixo_hidef_Nov17.edges", delim = "\t", col_names = FALSE)  %>%
subset(X3 == "gene") %>%
group_by(X1) %>% 
summarize(GeneList = paste0(X2, collapse = " ")) %>%
rowwise() %>%
mutate(n_Genes = length(str_split(string = GeneList, pattern = " ")[[1]])) %>%
subset(between(x = n_Genes, left = 3, right = 100)) %>%
mutate(Source = "NeST",
      GeneSetID = X1,
      GeneSetName = X1) %>%
select(-X1)

Rows: 39463 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): X1, X2, X3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [61]:
head(NeST_DF)

GeneList,n_Genes,Source,GeneSetID,GeneSetName
<chr>,<int>,<chr>,<chr>,<chr>
CTRL HSD17B14 KIAA0232 PAQR8 PLA2G1B RNF145 SGMS1 SLC7A8 TBC1D12 TMCC1 UBE2Q2 WDR47,12,NeST,Cluster1-10,Cluster1-10
LMF1 MFHAS1 MR1 PLA2G1B RASL11A RNF145 SLC2A6 SNX25 SPIRE1 TTPAL USP38 ZYG11B,12,NeST,Cluster1-11,Cluster1-11
AMY2B CNPY2 EGFL7 LDLR LPL LRP8 LRPAP1 MYLIP PLA2G1B RELN SORL1 VLDLR,12,NeST,Cluster1-12,Cluster1-12
ACO1 CDKAL1 CDX2 CIAO1 CITED2 FAM96A FAM96B FBXL5 IREB2 KDM5B MAFA MMS19 PCIF1 PDX1 RTEL1 TFAP2C,16,NeST,Cluster1-13,Cluster1-13
DTWD1 DTWD2 POLD2 POLD4 POLH POLI RAD18 REV1 REV3L TBC1D19 TTLL1 UBE2V1 UBE2V2 WRNIP1 ZNF611,15,NeST,Cluster1-14,Cluster1-14
BYSL CSNK1G3 DIMT1 RPS5 RPS13 RRP12,6,NeST,Cluster1-15,Cluster1-15


### SMALL Molecules

In [1]:
small_molecule_DF = read_delim("data/Omics_data/Small_molecule __gene_attribute_edges.txt", delim = "\t", skip = 1) %>%
group_by(`Drug Perturbation_Organism_GEO Platform_GEO Accession`) %>%
summarize(GeneList = paste0(GeneSym, collapse = " ")) %>%
rowwise() %>%
mutate(n_Genes = length(str_split(string = GeneList, pattern = " ")[[1]]))  %>%
subset(str_detect(string = `Drug Perturbation_Organism_GEO Platform_GEO Accession`, pattern = "homo sapiens")) %>%
subset(between(x = n_Genes, left = 3, right = 100)) %>%
mutate(Source = "Small Molecules",
       GeneSetID = `Drug Perturbation_Organism_GEO Platform_GEO Accession`,
       GeneSetName = `Drug Perturbation_Organism_GEO Platform_GEO Accession`) %>%
select(-"Drug Perturbation_Organism_GEO Platform_GEO Accession")


ERROR: Error in read_delim("data/Omics_data/Small_molecule __gene_attribute_edges.txt", : could not find function "%>%"


### Disease

In [25]:
pass = getPass::getPass("Enter the password: ")

Enter the password:  ········


In [26]:
disgenet_api_key <- get_disgenet_api_key(
                  email = "salkhairy@ucsd.edu", 
                  password = pass )

In [27]:
Sys.setenv(DISGENET_API_KEY= disgenet_api_key)

In [28]:
diseaseId_All_DF = read_delim("data/disease_associations.tsv", delim = "\t")


Rows: 30170 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (5): diseaseId, diseaseName, diseaseType, diseaseClass, diseaseSemanticType
dbl (2): NofGenes, NofPmids

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [29]:
diseaseId_All_list_names = diseaseId_All_DF$diseaseName
names(diseaseId_All_list_names) = diseaseId_All_DF$diseaseId

In [31]:
# Read disease list
diseaseId_All_list = diseaseId_All_DF %>%
subset(diseaseType == "disease") %>%
subset(between(x = NofGenes, left = 3, right = 100)) %>%
pull(diseaseId)

In [ ]:
# 'GWASCAT' to use the NHGRI-EBI GWAS Catalog; 

In [33]:
# Sample 
# diseaseId_list = sample(x = diseaseId_All_list, size = 500, replace = FALSE) # not all of them are successful in getting gene list

In [77]:
#disease_DF = genesets_empty_DF
an.error.occured <- FALSE
for (diseaseId_ind in c(7367:length(diseaseId_All_list))){
    print(diseaseId_ind)
    diseaseId = diseaseId_list[diseaseId_ind]
    diseaseName = diseaseId_All_list_names[diseaseId]
    
    tryCatch( { dis_res <- disease2gene( diseaseId_list[diseaseId_ind], database = "GWASCAT" ) } 
          , error = function(e) {an.error.occured <<- TRUE} )
    
    if (class(dis_res) == 'character'){next}
        
    geneList = dis_res@qresult %>% pull(gene_symbol)
    
    disease_DF = disease_DF %>% 
    add_row(tibble_row(Source = "Disease",
            GeneSetID = diseaseId,
            GeneSetName = diseaseName, 
            GeneList = str_c(geneList,  collapse = " "),
            n_Genes = length(geneList)))
    
    if (nrow(disease_DF) == 100) {break}
    }

[1] 7367
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



No encoding supplied: defaulting to UTF-8.



[1] "{\"errors\":{\"disease\":\"Disease/s not found in the database.\"},\"status_code\":404}"
[1] "no results for the query"
[1] 7368
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



No encoding supplied: defaulting to UTF-8.



[1] "{\"errors\":{\"disease\":\"Disease/s not found in the database.\"},\"status_code\":404}"
[1] "no results for the query"
[1] 7369
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



...

[1] "{\"errors\":{\"disease\":\"Disease/s not found in the database.\"},\"status_code\":404}"
[1] "no results for the query"
[1] 10667
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



No encoding supplied: defaulting to UTF-8.



[1] "{\"errors\":{\"disease\":\"Disease/s not found in the database.\"},\"status_code\":404}"
[1] "no results for the query"
[1] 10668
$category
[1] "Client error"

$reason
[1] "Not Found"

$message
[1] "Client error: (404) Not Found"



No encoding supplied: defaulting to UTF-8.



[1] "{\"errors\":{\"disease\":\"Disease/s not found in the database.\"},\"status_code\":404}"
[1] "no results for the query"


In [76]:
length(diseaseId_All_list)

[1] 10668

In [78]:
nrow(disease_DF)
disease_toUse_DF = disease_DF %>% 
subset(between(x = n_Genes, left = 3, right = 100)) %>%
head(n = 100)

[1] 15

In [79]:
nrow(disease_toUse_DF)

[1] 13

In [80]:
disease_toUse_DF = disease_toUse_DF %>%
rowwise() %>%
mutate(GeneSetName = ifelse(Source == "Disease", diseaseId_All_list_names[GeneSetID], GeneSetName))

In [81]:
write_delim(x = disease_toUse_DF,file = "data/disease_toUse_DF_revamped.txt" , delim = "\t")

### L1000

In [63]:
L1000_DF = read_delim("data/Omics_data/L1000__gene_attribute_edges.txt", delim = "\t", skip = 1) %>%
group_by(`Perturbation ID_Perturbagen_Cell Line_Time_Time Unit_Dose_Dose Unit`) %>%
summarize(GeneList = paste0(GeneSym, collapse = " ")) %>%
rowwise() %>%
mutate(n_Genes = length(str_split(string = GeneList, pattern = " ")[[1]]))  %>%
subset(between(x = n_Genes, left = 3, right = 100)) %>%
mutate(Source = "L1000",
       GeneSetID = `Perturbation ID_Perturbagen_Cell Line_Time_Time Unit_Dose_Dose Unit`,
       GeneSetName = `Perturbation ID_Perturbagen_Cell Line_Time_Time Unit_Dose_Dose Unit`) %>%
select(-"Perturbation ID_Perturbagen_Cell Line_Time_Time Unit_Dose_Dose Unit")

New names:
• `NA` -> `NA...2`
• `NA` -> `NA...5`
• `NA` -> `NA...6`
Rows: 4189677 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): GeneSym, NA...2, Perturbation ID_Perturbagen_Cell Line_Time_Time Un...
dbl (3): GeneID, NA...6, weight

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [72]:
genesets_test_DF = genesets_empty_DF %>%
rbind(head(NeST_DF, 10)) %>%
rbind(head(small_molecule_DF, 10)) %>%
rbind(head(disease_toUse_DF, 10)) %>%
rbind(head(L1000_DF, 10))

In [73]:
dim(genesets_test_DF)

[1] 40  5

In [ ]:
test

In [ ]:
write_delim(x = genesets_DF,file = "data/omics_revamped.txt" , delim = "\t")

In [74]:
write_delim(x = genesets_test_DF,file = "data/omics_test_revamped.txt" , delim = "\t")

# NOT CLEAN AFTER THIS

### Viral

In [ ]:
viral_Infections_DF = read_delim("data/Omics_data/Viral_Infections__gene_attribute_edges.txt", delim = "\t", skip = 1) %>%
group_by(`Virus Perturbation`) %>%
summarize(GeneList = paste0(GeneSym, collapse = " ")) %>%
rowwise() %>%
mutate(n_Genes = length(str_split(string = GeneList, pattern = " ")[[1]])) %>%
subset(between(x = n_Genes, left = 3, right = 100)) %>%
mutate(Source = "Viral Infections",
       GeneSetID = "Virus Perturbation",
       GeneSetName = "Virus Perturbation") %>%
select(-"Virus Perturbation")


In [ ]:
# subset(str_detect(string = Duration, pattern = "24Hour|Day1|1day")) %>%
# separate(col = `Virus Perturbation`, into = c("Virus", "Duration", "Location"), sep = "_", remove = FALSE) %>%
